*Disclaimer: This work is a giant pile of scratch work, and is not very organized.*

# Notes
- My models appear to be okay, but my preprocessing still leaves something to be desired.
- With my method of preprocessing, I am able to get close to 0.80 accuracy with random forests, but I am routinely around 0.70 on SVC despite assertions everywhere that SVC should get me past 0.80.
- I tried preprocessing code from https://www.kaggle.com/sinakhorami/titanic-best-working-classifier, still on my own models, but was still only around 0.78 on Kaggle.

In [1]:
#!/usr/bin/env python

# (c) 2017, Chris Hodapp
# Code for: https://www.kaggle.com/c/titanic

# "It is your job to predict if a passenger survived the sinking of
# the Titanic or not. For each PassengerId in the test set, you must
# predict a 0 or 1 value for the Survived variable."  "Your score is
# the percentage of passengers you correctly predict."

import pandas
import matplotlib
import matplotlib.pyplot as plt
import numpy
import xgboost
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.preprocessing
import sklearn.ensemble
import sklearn.svm

/nix/store/52jgbsm6l4145pl8k9d5z2nj7ss4v35a-python3-3.5.4-env/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Another person's preprocessing code, for comparison
import sinakhorami_preprocess

/home/hodapp/source/kaggle-titanic/sinakhorami_preprocess.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list


## Loading & Transformation:

In [3]:
pandas.set_option("display.width", None)

# Load data:
train_raw = pandas.read_csv("train.csv")
test_raw = pandas.read_csv("test.csv")

In [4]:
# Some titles are equivalent:
equiv = {
    "Jonkheer": "Master",
    "Mlle": "Miss",
    "Ms": "Miss",
    "Mme": "Mrs",
    "Capt": "Sir",
    "Don": "Sir",
    "Major": "Sir",
    "Col": "Sir",
    "Dona": "Lady",
    "Countess": "Lady",
}
def getprefix(name):
    prefix = [pt[:-1] for pt in name.split(" ") if pt.endswith(".")][0]
    return equiv.get(prefix, prefix)

def xform_input(df):
    # One-hot encode categorical Embarked column
    # Turn column "Sex" to a number - 0 for male, 1 for female
    # (it has only these two categories, all non-null)
    # Extract first letter of Cabin and one-hot encode
    # Extract title (Mr, Mrs, Mme, Captain, etc.) from Title, one-hot encode
    onehot_emb = pandas.get_dummies(df.Embarked, "Embarked")
    onehot_cabin = pandas.get_dummies([i[0] for i in df.Cabin.fillna("N")], "Cabin")
    onehot_title = pandas.get_dummies([getprefix(n) for n in df.Name if "." in n], "Title")
    counts = train_raw.groupby("Ticket", as_index=False).size().reset_index(name="Count")
    df2 = train_raw.merge(counts, on="Ticket")
    return df.assign(Sex = numpy.where(df.Sex == "male", 0, 1)) \
             .assign(FarePer = df2.Fare / df2.Count) \
             .assign(Alone = ((df2.SibSp + df2.Parch) == 0)) \
             .drop("Embarked", axis=1) \
             .drop("Cabin", axis=1) \
             .join(onehot_cabin) \
             .join(onehot_emb) \
             .join(onehot_title)

## Raw data:

In [5]:
train_raw.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Feature selection:

In [6]:
# Do some initial transformation:
train = xform_input(train_raw)

# Select columns ('prefix_cols' grabs entire group by prefix):
prefix_cols = ["Embarked", "Title"] #, "Cabin"]
def matches_prefix(col):
    for p in prefix_cols:
        if col.startswith(p): return True
    return False
cols = ["Sex", "Age", "Fare", "SibSp", "Parch"] + \
    [i for i in train.columns if matches_prefix(i)]
# ["FarePer", "Count", "Pclass"] \
train_X = train.loc[:, cols]
train_Y = train.loc[:, "Survived"]

test = xform_input(test_raw)
test.loc[:,"Cabin_T"] = 0
test_X = test.loc[:, cols]

## Missing data:

In [7]:
# Fare has just one missing value to take care of
# (for which we just fill in the average):
fare_avg = (train_X.Fare.sum() + test_X.Fare.sum()) / (train_X.Fare.count() + test_X.Fare.count())
test_X.Fare.fillna(fare_avg, inplace=True)

# Age has a lot more; split apart the missing Age rows
# from both training & test:
mask_train = train_X.Age.isnull()
mask_test = test_X.Age.isnull()
age_valid = pandas.concat([train_X[~mask_train],  test_X[~mask_test]])
age_valid_Y = age_valid.Age
age_valid_X = age_valid.drop("Age", axis=1)
train_age_null = train_X[mask_train].drop("Age", axis=1)
test_age_null = test_X[mask_test].drop("Age", axis=1)

In [8]:
age_valid.shape[0], train_age_null.shape[0], test_age_null.shape[0]

(1046, 177, 86)

In [9]:
train_age_null.head(5)

,Sex,Fare,SibSp,Parch,Embarked_C,Embarked_Q,Embarked_S,Title_Dr,Title_Lady,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rev,Title_Sir
5,0,8.4583,0,0,0,1,0,0,0,0,0,1,0,0,0
17,0,13.0000,0,0,0,0,1,0,0,0,0,1,0,0,0
19,1,7.2250,0,0,1,0,0,0,0,0,0,0,1,0,0
26,0,7.2250,0,0,1,0,0,0,0,0,0,1,0,0,0
28,1,7.8792,0,0,0,1,0,0,0,0,1,0,0,0,0


In [10]:
age_valid_X.isnull().sum()

Sex             0
Fare            0
SibSp           0
Parch           0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
Title_Dr        0
Title_Lady      0
Title_Master    0
Title_Miss      0
Title_Mr        0
Title_Mrs       0
Title_Rev       0
Title_Sir       0
dtype: int64

In [11]:
# Replace missing values of Age based on other values.
# Since this is an unrelated model (not the one we are predicting
# Survived from), we aren't tainting it by including test data in it.
rfr = sklearn.ensemble.RandomForestRegressor()
rfr = rfr.fit(age_valid_X, age_valid_Y)

In [12]:
train_X.loc[mask_train, "Age"] = rfr.predict(train_age_null)
test_X.loc[mask_test, "Age"]   = rfr.predict(test_age_null)

## Transformed data:

In [13]:
train_X.head(5)

,Sex,Age,Fare,SibSp,Parch,Embarked_C,Embarked_Q,Embarked_S,Title_Dr,Title_Lady,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rev,Title_Sir
0,0,22.0,7.2500,1,0,0,0,1,0,0,0,0,1,0,0,0
1,1,38.0,71.2833,1,0,1,0,0,0,0,0,0,0,1,0,0
2,1,26.0,7.9250,0,0,0,0,1,0,0,0,1,0,0,0,0
3,1,35.0,53.1000,1,0,0,0,1,0,0,0,0,0,1,0,0
4,0,35.0,8.0500,0,0,0,0,1,0,0,0,0,1,0,0,0


## Models to test

In [18]:
# Make a zoo of models to test:
logistic = sklearn.linear_model.LogisticRegression(C = 1e5)
rf = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100, criterion="entropy", max_depth=5, n_jobs=-1,
    random_state=12348)
rf2 = sklearn.ensemble.RandomForestClassifier(
    n_estimators=200, criterion="entropy", max_depth=5, n_jobs=-1,
    random_state=12348)
rf3 = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100, criterion="entropy", max_depth=10, n_jobs=-1,
    random_state=12348)
rf4 = sklearn.ensemble.RandomForestClassifier(
    n_estimators=200, criterion="entropy", max_depth=10, n_jobs=-1,
    random_state=12348)
ab = sklearn.ensemble.AdaBoostClassifier(
    n_estimators=400,
    learning_rate=1.0,
    random_state=12345)
svc = sklearn.svm.SVC()
xgb = xgboost.sklearn.XGBClassifier(nthread=-1, seed=12345)

models = {
    "Logistic regression": logistic,
    "Random forest(100,5)": rf,
    "Random forest(200,5)": rf2,
    "Random forest(100,10)": rf3,
    "Random forest(200,10)": rf4,
    "AdaBoost": ab,
    "SVC": svc,
    "XGBoost": xgb
}

### My preprocessing & models:

In [19]:
data = []
for name in models:
    scores = sklearn.model_selection.cross_val_score(
        models[name],
        train_X,
        train_Y,
        cv=sklearn.model_selection.StratifiedShuffleSplit())
    data.append((name, scores.mean(), scores.std() * 2))
data = pandas.DataFrame.from_records(
    data, columns=("Model", "Mean Accuracy", "Std. Dev."))
data.sort_values("Model", inplace=True)

In [20]:
data

,Model,Mean Accuracy,Std. Dev.
2,AdaBoost,0.798889,0.049141
3,Logistic regression,0.787778,0.077268
1,"Random forest(100,10)",0.820000,0.047454
5,"Random forest(100,5)",0.821111,0.077905
0,"Random forest(200,10)",0.827778,0.075195
6,"Random forest(200,5)",0.831111,0.058626
4,SVC,0.755556,0.062063
7,XGBoost,0.834444,0.092403


### My models, but sinakhorami's features:

In [ ]:
data = []
for name in models:
    scores = sklearn.model_selection.cross_val_score(
        models[name],
        sinakhorami_preprocess.train[:,1::],
        sinakhorami_preprocess.train[:,0],
        cv=sklearn.model_selection.StratifiedShuffleSplit())
    data.append((name, scores.mean(), scores.std() * 2))
data = pandas.DataFrame.from_records(
    data, columns=("Model", "Mean Accuracy", "Std. Dev."))
data.sort_values("Model", inplace=True)

In [ ]:
data

In [ ]:
%matplotlib inline
matplotlib.pyplot.figure()
xgb.fit(train_X, train_Y)
xgboost.plot_importance(xgb, ax=matplotlib.pyplot.gca())

In [26]:
def plot_validation(train_scores, test_scores, param_range, linear=False):
    # Code adapted from:
    # http://scikit-learn.org/stable/auto_examples/model_selection/plot_validation_curve.html
    train_scores_mean = numpy.mean(train_scores, axis=1)
    train_scores_std = numpy.std(train_scores, axis=1)
    test_scores_mean = numpy.mean(test_scores, axis=1)
    test_scores_std = numpy.std(test_scores, axis=1)
    plt.ylabel("Score")
    #plt.ylim(0.0, 1.1)
    lw = 2
    if linear:
        plt.plot(param_range, train_scores_mean, label="Training score",
                 color="darkorange", lw=lw)
    else:
        plt.semilogx(param_range, train_scores_mean, label="Training score",
                     color="darkorange", lw=lw)
    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2,
                     color="darkorange", lw=lw)
    if linear:
        plt.plot(param_range, test_scores_mean, label="Cross-validation score",
                 color="navy", lw=lw)
    else:
        plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
                     color="navy", lw=lw)
    plt.fill_between(param_range, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.2,
                     color="navy", lw=lw)
    plt.legend(loc="best")
    plt.show()

## Random Forest experimentation

In [22]:
rf = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100, criterion="entropy", max_depth=5, n_jobs=-1,
    random_state=12348)
rf = rf.fit(train_X, train_Y)
features = pandas.DataFrame(
    {"Feature": list(train_X.columns),
     "Importance": rf.feature_importances_})
features.sort_values("Importance", inplace=True, ascending=False)
features.head(15)

,Feature,Importance
12,Title_Mr,0.272764
0,Sex,0.181875
2,Fare,0.174248
1,Age,0.080788
3,SibSp,0.073116
13,Title_Mrs,0.064942
11,Title_Miss,0.058950
4,Parch,0.029853
5,Embarked_C,0.018746
10,Title_Master,0.014316


In [28]:
def test_estimators(est_range, max_depth):
    train_scores, valid_scores = sklearn.model_selection.validation_curve(
        sklearn.ensemble.RandomForestClassifier(
            criterion="entropy", max_depth=max_depth, n_jobs=-1,
            random_state=12348),
        train_X, train_Y,
        "n_estimators", est_range, cv=sklearn.model_selection.StratifiedShuffleSplit())
    plt.title("Random Forest Classifier, max_depth={0}".format(max_depth))
    plt.xlabel("n_estimators")
    plot_validation(train_scores, valid_scores, est_range)

In [ ]:
rng = numpy.logspace(0.5, 3.5, 10).astype(numpy.int64)
test_estimators(rng, 5)

In [ ]:
test_estimators(rng, 7)

In [ ]:
test_estimators(rng, 9)

In [ ]:
test_estimators(rng, 11)

## SVM validation curves

In [ ]:
rng = numpy.logspace(-3, -1, 30)
train_scores, valid_scores = sklearn.model_selection.validation_curve(
    sklearn.svm.SVC(C=1, probability=True), train_X, train_Y,
    "gamma", rng, cv=sklearn.model_selection.StratifiedShuffleSplit())
plt.title("SVC validation curve, C=1")
plt.xlabel("$\gamma$")
plot_validation(train_scores, valid_scores, rng)

In [ ]:
rng = numpy.logspace(-1, 2, 30)
train_scores, valid_scores = sklearn.model_selection.validation_curve(
    sklearn.svm.SVC(gamma=0.03, probability=True), train_X, train_Y,
    "C", rng, cv=sklearn.model_selection.StratifiedShuffleSplit())
plt.title("SVC validation curve, $\gamma$=0.03")
plt.xlabel("C")
plot_validation(train_scores, valid_scores, rng)

## xgboost experimentation

In [ ]:
xgb = xgboost.sklearn.XGBClassifier(nthread=-1, seed=12345)
rng = numpy.logspace(-2, 0, 30)
train_scores, valid_scores = sklearn.model_selection.validation_curve(
    xgb, train_X, train_Y,
    "learning_rate", rng, cv=sklearn.model_selection.StratifiedShuffleSplit())
plt.title("xgboost validation curve")
plt.xlabel("$\eta$")
plot_validation(train_scores, valid_scores, rng)

In [ ]:
xgb = xgboost.sklearn.XGBClassifier(nthread=-1, seed=12345, learning_rate=0.1)
rng = numpy.logspace(1.5, 3.5, 30).astype(numpy.int64)
train_scores, valid_scores = sklearn.model_selection.validation_curve(
    xgb, train_X, train_Y,
    "n_estimators", rng, cv=sklearn.model_selection.StratifiedShuffleSplit())
plt.title("xgboost validation curve, $\eta=0.1$")
plt.xlabel("n_estimators")
plot_validation(train_scores, valid_scores, rng)

In [ ]:
xgb = xgboost.sklearn.XGBClassifier(nthread=-1, seed=12345, learning_rate=0.1, n_estimators=200)
rng = numpy.logspace(-3, 2, 30)
train_scores, valid_scores = sklearn.model_selection.validation_curve(
    xgb, train_X, train_Y,
    "reg_lambda", rng, cv=sklearn.model_selection.StratifiedShuffleSplit())
plt.title("xgboost validation curve, $\eta=0.1$, n_estimators=200")
plt.xlabel("$\lambda$")
plot_validation(train_scores, valid_scores, rng)

In [ ]:
xgb = xgboost.sklearn.XGBClassifier(nthread=-1, seed=12345, learning_rate=0.1, n_estimators=200)
rng = numpy.arange(1, 15)
train_scores, valid_scores = sklearn.model_selection.validation_curve(
    xgb, train_X, train_Y,
    "max_depth", rng, cv=sklearn.model_selection.StratifiedShuffleSplit())
plt.title("xgboost validation curve, $\eta=0.1$, n_estimators=200")
plt.xlabel("Max depth")
plot_validation(train_scores, valid_scores, rng, linear=True)

In [ ]:
xgb = xgboost.sklearn.XGBClassifier(nthread=-1, seed=12345, learning_rate=0.1, n_estimators=200, max_depth=6)
rng = numpy.linspace(0, 4, 25)
train_scores, valid_scores = sklearn.model_selection.validation_curve(
    xgb, train_X, train_Y,
    "gamma", rng, cv=sklearn.model_selection.StratifiedShuffleSplit())
plt.title("xgboost validation curve, $\eta=0.1$, n_estimators=200, depth=6")
plt.xlabel("$\gamma$")
plot_validation(train_scores, valid_scores, rng, linear=True)

In [ ]:
xgb = xgboost.sklearn.XGBClassifier(nthread=-1, seed=12345, learning_rate=0.1, n_estimators=400, max_depth=6)
rng = numpy.linspace(0, 4, 25)
train_scores, valid_scores = sklearn.model_selection.validation_curve(
    xgb, train_X, train_Y,
    "gamma", rng, cv=sklearn.model_selection.StratifiedShuffleSplit())
plt.title("xgboost validation curve, $\eta=0.1$, n_estimators=400, depth=6")
plt.xlabel("$\gamma$")
plot_validation(train_scores, valid_scores, rng, linear=True)

## Submission code

In [ ]:
# Generate submission:
model = sklearn.ensemble.RandomForestClassifier(
        n_estimators=400, criterion="entropy", max_depth=5, n_jobs=-1)
#model = xgboost.sklearn.XGBClassifier(nthread=-1, seed=12346, learning_rate=0.1, n_estimators=400, max_depth=6, gamma=0.5)
#model = sklearn.svm.SVC()
#model.fit(sinakhorami_preprocess.train[:,1::],
#          sinakhorami_preprocess.train[:,0],
#         )
model.fit(train_X, train_Y)
submission = test[["PassengerId"]]
#submission = submission.assign(Survived = model.predict(sinakhorami_preprocess.test))
submission = submission.assign(Survived = model.predict(test_X))
submission.to_csv("submission.csv", index=False)